In [ ]:
import json
import os

import openai
from langchain.tools import OpenAPISpec
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
MP_API_KEY = os.getenv("MP_API_KEY", None)

spec = OpenAPISpec.from_file("mp_openapi.json")

In [ ]:
from openapi_schema_pydantic import PathItem, Operation, Parameter

# TODO: construct a class


def get_endpoints(spec: OpenAPISpec):
    endpoints = [
        (route, method, operation)
        for route, paths in spec.paths.items()
        for method, operation in paths
        if method in ["get", "post"]
    ]
    return endpoints


def get_functions(spec: OpenAPISpec):
    endpoints = get_endpoints(spec)
    functions = []
    for route, method, operation in endpoints:
        if operation is None:
            continue

        name = (
            operation.operationId
            if len(operation.operationId) <= 64
            else operation.operationId[:64]
        )

        description = operation.description or operation.summary

        properties = {
            property.name: {
                "type": property.param_schema.type,
                "description": property.description
                if property.description is not None
                else "",
                **(
                    {"enum": property.param_schema.enum}
                    if property.param_schema.enum is not None
                    else {}
                ),
            }
            for property in operation.parameters
            if property.param_schema.type is not None
        }

        required = [
            property.name for property in operation.parameters if property.required
        ]
        functions.append(
            {
                "name": name,
                "description": description,
                "parameters": {
                    "type": "object",
                    "properties": properties,
                },
                **({"required": required} if required else {}),
            }
        )

    return functions


functions = get_functions(spec)

print(len(functions))

In [ ]:
functions[0]

In [ ]:
functions[11]

In [ ]:
# functions = [functions[11]]
# functions = [functions[0]]
functions

In [ ]:
user_input = "What is the bond length of H2O?"

messages = [{"role": "user", "content": user_input}]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response
# response_message = response["choices"][0]["message"]

In [ ]:
response_message

In [ ]:
spec.paths["/mpcomplete/"].get.parameters